# Image Text Extraction and Information Retrieval

## The Question

Being able to take pictures of text, recognize, convert into computer text, and search either typed or hand-written is a very interesting and useful concept.
**What is the best way to extract text from images and search through them with specific queries?**

## The Data

Our data consists of **typed and hand-written document images and scans**. We created a dataset that is combined from multiple other datasets. It consists of approximately **1509 data points.**

## The Approach

Since all these images have different document styles and structures, we want to see how well the image text detection library we're using will be able to convert the document into computer text thus allowing us to search through the documents. We will not be implementing the document image word extraction or TF-IDF from scratch, but we will be using multiple libraries in order to see how these machine learning and natural language processing concepts and libraries work. The libraries we will be using are

## Our Hypothesis



## Why Bother?

Teaching a machine to recognize physical typed and hand-written documents from pictures and scans as well as being able to search through these documents is a very important and useful machine learning concept.

There are many applications such as being able to scan these documents into a database and being able to search for information within the documents.

## Basic Setup

### PIL, Pytesseract

We first loaded in the machine learning libraries. **PIL** allows us to process the images. **Pytesseract** allows us to convert the text in the image into a text string.

In [4]:
!pip install pytesseract
!pip install googletrans
!pip install pytesseract
!pip install pandas

In [8]:
# adds image processing capabilities
from PIL import Image

# will convert the image text into a text string
import pytesseract

import os

import pandas as pd

from elasticsearch import Elasticsearch, helpers
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
import csv
import sys

## Convert Documents

In [5]:
#list for storing each image and it's extracted text
images = []
results = []
amount = 0

for filename in os.listdir('Data/Typed'):
    if filename.endswith('.png'):
        with open(os.path.join('Data/Typed', filename)) as f:
            # assigning an image from the source path
            img = Image.open('Data/Typed/' + filename)
            images.append(img)
            # converts the image to result and saves it into result variable
            result = pytesseract.image_to_string(img)
            results.append(result)
            amount += 1
            print(amount)

for filename in os.listdir('Data/Written'):
    if filename.endswith('.jpg'):
        with open(os.path.join('Data/Written', filename)) as f:
            # assigning an image from the source path
            img = Image.open('Data/Written/' + filename)
            images.append(img)
            # converts the image to result and saves it into result variable
            result = pytesseract.image_to_string(img)
            results.append(result)
            amount += 1
            print(amount)
    if filename.endswith('.png'):
        with open(os.path.join('Data/Written', filename)) as f:
            # assigning an image from the source path
            img = Image.open('Data/Written/' + filename)
            images.append(img)
            # converts the image to result and saves it into result variable
            result = pytesseract.image_to_string(img)
            results.append(result)
            amount += 1
            print(amount)

KeyboardInterrupt: 

## Store in Datastore

In [ ]:
df = pd.DataFrame({
        'image': images,
        'result': results
})

df.to_csv('output.csv', index=False)

## Elasticsearch

In [7]:
with open('output.csv') as f:
    reader = csv.DictReader(f)
    es.indices.delete(index='documents')
    helpers.bulk(es, reader, index='documents')

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x1238faa90>: Failed to establish a new connection: [Errno 61] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x1238faa90>: Failed to establish a new connection: [Errno 61] Connection refused)

In [ ]:
searching = True

while searching:
    search = input("Search (to exit type 'exit'): ")

    if (search == 'exit') or (search == 'Exit'):
        break
    else:
        result = es.search(
            index="documents",
            body={
                "query": {
                    "multi_match": {
                        "query": search,
                        "fields": ["result"]
                    }
                }
            }
        )
    all_hits = result['hits']['hits']

    for hit in all_hits:
        print("Result:", hit['_source']['result'])
        print("Image:", hit['_source']['image'])

## The Data

## Voila!